# Prueba - Data Engineer

# Prueba Teórica

1. ¿Qué es un sistema distribuido ?
    - Un sistema distribuido es un conjunto de equipos, los cuales trabajan de forma independiente y coordinada como un único equipo. Realizan intercambio de mensajes para conseguir un objetivo.
    
2. Nombre sus principales ventajas
    - Velocidad en el procesamiento, dado que cuando se realiza una consulta, los procesamientos se dividen entre todos los nodos.
    - Mayor tolerancia a fallos: Al ser independiente del fallo de los componentes, el hecho de que un nodo falle, no lleva al fallo de todo el sistema

3. Nombre 2 diferencias con los sistemas centralizados
    - El sistema centralizado, no trabaja con un conjunto de equipos para obtener un objetivo sino que trabajo con un único equipo.
    - El sistema centralizado ofrece mayor seguridad porque todo el procesamiento es controlado en una localización central.
  
  
4. Describa brevemente el funcionamiento de un sistema de bases de datos distribuido configurado como Sharded Cluster.
    - El sharded cluster contiene los siguientes componentes:
        - *shard*: Cada shard contiene un subconjunto de los datos sharded.
        - *mongos*: funciona como un router de query, y es la interfaz entre el cliente y el cluster sharded.
        - *config servers*: almacena datos y configura los clusters.
    - Se tiene de forma central el *config servers* el cual se conecta con el o los router de mongos y tambien con 2 o mas shards. Del mismo modo los routers tambien se conectan con los shards.
5. Nombre al menos 2 operaciones de transformación y 2 operaciones de acción que sean ejecutables en un RDD con spark, cite un ejemplo de cada una.
    - Spark soporta dos tipos de transformaciones sobre RDD:

        - Transformaciones: Son funciones que crean un nuevo RDD a partir de uno existente. Estas pueden entenderse como equivalentes a la función Map en el algorimto MapReduce.
            - join( otherRDD ): para los RDD (K, V) y (K, W) retorna (K, (V, W)).
            - aggregateByKey(): agrega los elementos por clave.
        - Acciones: Retornan un valor después de realizar una computación sobre el RDD. Pueden entenderse como un reducer.
            - count(): Retorna el número de elementos del RDD.
            - reduce( f ): agrega los elementos con la misma clave usando f.

6. ¿Cuál es la principal diferencia entre Cloud Run y Cloud Function en el sistema Google Cloud Platform? Cite un ejemplo de implementación en cada una de las soluciones.

    - Cloud Run toma los "containers" y los hace invocables por medio de solicitudes de HTTP.
    - Cloud Function, es una plataforma de google sin servidor la cual es imporsada por eventos de google.  A diferencia de estar empaquetado en un "container" de Docker, el código se implementa como funciones.

# Ejercicio Práctico

In [1]:
import pandas as pd
import numpy as np

## Cargar Base

In [2]:
df_ads = pd.read_csv('Ads schedule.csv')
df_ans = pd.read_csv('Analytics.csv')

C:\Users\Natalia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_ans.head()

,date,source,medium,campaign,adContent,clientId,operatingSystem,browser,language,country,city,sessionQualityDim,newVisits,pageviews,bounces
0,1082016,youtube.com,referral,(not set),NaN,NaN,Windows,Internet Explorer,not available in demo dataset,United States,not available in demo dataset,NaN,1.0,3.0,NaN
1,1082016,youtube.com,referral,(not set),NaN,NaN,Windows,Chrome,not available in demo dataset,United States,San Jose,NaN,NaN,3.0,NaN
2,1082016,youtube.com,referral,(not set),NaN,NaN,Macintosh,Chrome,not available in demo dataset,Philippines,Quezon City,NaN,1.0,3.0,NaN
3,1082016,youtube.com,referral,(not set),NaN,NaN,Windows,Firefox,not available in demo dataset,Canada,not available in demo dataset,NaN,1.0,4.0,NaN
4,1082016,youtube.com,referral,(not set),NaN,NaN,Windows,Internet Explorer,not available in demo dataset,United States,not available in demo dataset,NaN,1.0,2.0,NaN


In [4]:
df_ads.head()

,_id,medium,support,reference,coverage,start_date,end_date,format,inversion,rating,channel,calculated,date_loaded
0,5ed306339be6f111072434b7,Medio_A,CUENTAME,C. Donación,FR,2020-05-19 13:21:49 UTC,2020-05-19 13:22:49 UTC,Mención 60,85.991836,0.0,2CAN+1,True,2020-05-31 01:19:47 UTC
1,5ed306339be6f11107242e49,Medio_A,CUENTAME,C. Restaurantes,FR,2020-05-14 13:57:53 UTC,2020-05-14 13:59:04 UTC,Mención 60,85.991836,0.0,2CAN+1,True,2020-05-31 01:19:47 UTC
2,5ed306339be6f11107242e2d,Medio_A,CUENTAME,C. Farmacia,FR,2020-05-11 13:57:59 UTC,2020-05-11 13:59:02 UTC,Mención 60,85.991836,0.0,2CAN+1,True,2020-05-31 01:19:47 UTC
3,5ed5d8ffc486d565172eae7a,Medio_A,CUENTAME,C. Pañales,FR,2020-05-27 14:03:35 UTC,2020-05-27 14:05:00 UTC,Mención 60,85.991836,0.0,2CAN+1,True,2020-06-02 04:43:43 UTC
4,5ed306339be6f11107243666,Medio_A,CUENTAME,D. de tema,FR,2020-05-20 14:00:21 UTC,2020-05-20 14:03:40 UTC,3 a 4 min,129.046442,0.0,2CAN+1,True,2020-05-31 01:19:47 UTC


## Completitud de los datos

El código realizado es en python, si se realiza en SQL se realizaría de la siguiente manera:
1. Se obtiene el nombre de cada una de la columnas con INFORMATION_SCHEMA.COLUMNS y se guardan los nombres de las columnas

    ```SELECT DATA_TYPE 
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE 
         TABLE_NAME = 'yourTableName'```
         
2. Se recorre cada columna, se cuenta la cantidad de registros, se filtran las columnas donde sean diferente de NULL:

    ``` SELECT COUNT(*) 
    FROM TABLE 
    WHERE COLUMNA IS NOT NULL```
    
3. Se crea una tabla que contiene las columnas de la tabla es una fila y en la otra la cantidad de registros con datos.

    ```CREATE TABLE table_name (
    Metrica varchar(100),
    cant_val int);```

4. El select realizado en el punto 2 se inserta en en la tabla creada
    
    ```INSERT INTO table_name(
    SELECT @col_name,COUNT(*) 
    FROM TABLE 
    WHERE COLUMNA IS NOT NULL)```


In [5]:
def df_isnull(df):
    columns = list(df)
    comp = pd.DataFrame(columns =['metrica','cant_comp','cant_null'])
    for i in columns:
        comp = comp.append({'metrica' : i, 'cant_comp' : df.shape[0] - df[i].isna().sum(),
                            'cant_null' : df[i].isna().sum()},ignore_index = True) 
    return comp

In [6]:
comp_analysis = df_isnull(df_ans)

En la tabla **Analytics** se puede observar:
- Las métricas que no cumplen con la completitud de los datos son:
    - `adContent`
    - `clientId`
    - `sessionQualityDim`	
    - `newVisits`		
    - `pageviews`
    - `bounces`

- También se observa que la métrica `clientId` no contiene ningún valor. 

In [7]:
comp_analysis

,metrica,cant_comp,cant_null
0,date,908148,0
1,source,908148,0
2,medium,908148,0
3,campaign,908148,0
4,adContent,10956,897192
5,clientId,0,908148
6,operatingSystem,908148,0
7,browser,908148,0
8,language,908148,0
9,country,908148,0


In [8]:
comp_ads = df_isnull(df_ads)

En la tabla **Ads schedule** se puede observar:
- Las métricas que no cumplen con la completitud de los datos son:
    - `start_date`
    - `inversion`

In [9]:
comp_ads

,metrica,cant_comp,cant_null
0,_id,76231,0
1,medium,76231,0
2,support,76231,0
3,reference,76231,0
4,coverage,76231,0
5,start_date,75229,1002
6,end_date,76231,0
7,format,76231,0
8,inversion,74696,1535
9,rating,76231,0


## Duplicidad de los datos

El código realizado es en python, si se realiza en SQL se realizaría de la siguiente manera:
1. Se realiza un distinct de todas las columnas y luego se realiza un conteo al resultado en la variable creada:

    ```declare @cant_unique int```
    
    ```set @cant_unique = (SELECT COUNT(*) FROM (SELECT DISTINCT * FROM TABLE))```
    

2. Del valor obtenido en el punto anterior se resta al total de los datos:
    
    ```declare @total int```
    
    ```set @total = (SELECT COUNT(*) FROM TABLE)```
    
    ```declare @cant_dup int```
    
    ```set @cant_dup = @total - @cant_unique```

En `@cant_dup` se tendría la cantidad de registros duplicados


In [10]:
def duplicate_df(df):
    df_dup = df[df.duplicated()].copy()
    df_dup.reset_index
    cant_dup = df_dup.shape[0]
    return df_dup,cant_dup

In [11]:
df_ans_dup, cant_ans_dup = duplicate_df(df_ans)
print(cant_ans_dup)

301268


In [12]:
df_ans_dup.head()

,date,source,medium,campaign,adContent,clientId,operatingSystem,browser,language,country,city,sessionQualityDim,newVisits,pageviews,bounces
76,1082016,(direct),(none),(not set),NaN,NaN,Macintosh,Chrome,not available in demo dataset,United States,Mountain View,NaN,1.0,2.0,NaN
83,1082016,(direct),(none),(not set),NaN,NaN,Macintosh,Chrome,not available in demo dataset,United States,New York,NaN,NaN,2.0,NaN
84,1082016,(direct),(none),(not set),NaN,NaN,Windows,Chrome,not available in demo dataset,United States,Salem,NaN,NaN,2.0,NaN
102,1082016,(direct),(none),(not set),NaN,NaN,Android,Chrome,not available in demo dataset,United States,not available in demo dataset,NaN,1.0,2.0,NaN
110,1082016,google,organic,(not set),NaN,NaN,Macintosh,Chrome,not available in demo dataset,United States,not available in demo dataset,NaN,NaN,3.0,NaN


In [13]:
df_ads_dup, cant_ads_dup = duplicate_df(df_ads)
print(cant_ads_dup)

0


In [14]:
df_ads_dup.head()

,_id,medium,support,reference,coverage,start_date,end_date,format,inversion,rating,channel,calculated,date_loaded


De lo anterior se puede observar que la base **Analytics** tiene 301268 registros repetidos, mientras que la base **Ads schedule** no tiene ningun valor repetido.

## Consistencia de los Datos

El código realizado es en python, si se realiza en SQL se realizaría de la siguiente manera:

1. Se obtienen los tipos de las columnas con `INFORMATION_SCHEMA.COLUMNS`:
    ```SELECT DATA_TYPE 
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = 'yourTableName'```
2. Se realiza el mismo ejercicio  con las dos bases de datos, y se observa el tipo de datos de las columnas o métricas en común.
3. En caso que sean distintas, se cambian por medio de la funcion `CAST(COLOLUMN AS DATA_TYPE)` o `CONVERT(DATA_TYPE, COLUMN)`

In [15]:
df_ans.dtypes

date                  object
source                object
medium                object
campaign              object
adContent             object
clientId             float64
operatingSystem       object
browser               object
language              object
country               object
city                  object
sessionQualityDim    float64
newVisits            float64
pageviews            float64
bounces              float64
dtype: object

In [16]:
df_ads.dtypes

_id             object
medium          object
support         object
reference       object
coverage        object
start_date      object
end_date        object
format          object
inversion      float64
rating         float64
channel         object
calculated        bool
date_loaded     object
dtype: object

In [17]:
df_ans['date'].unique()

array([1082016, 20160802, 20160803, 20160804, 20160805, 20160806,
       20160807, 20160808, 20160809, 20160810, 20160811, 20160812,
       20160813, 20160814, 20160815, 20160816, 20160817, 20160818,
       20160819, 20160820, 20160821, 20160822, 20160823, 20160824,
       20160825, 20160826, 20160828, 20160829, 20160830, 20160831,
       20160901, 20160902, 20160903, 20160904, 20160905, 20160906,
       20160907, 20160908, 20160909, 20160910, 20160911, 20160912,
       20160913, 20160914, 20160915, 20160916, 20160917, 20160918,
       20160919, 20160920, 20160921, 20160922, 20160923, 20160924,
       20160925, 20160926, 20160927, 20160928, 20160929, 20160930,
       20161001, 20161002, 20161003, 20161004, 20161005, 20161006,
       20161007, 20161008, 20161009, 20161010, 20161011, 20161012,
       20161013, 20161014, 20161015, 20161016, 20161017, 20161018,
       20161019, 20161020, 20161021, 20161022, 20161023, 20161024,
       20161025, 20161026, 20161027, 20161028, 20161029, 201610

En este caso se puede observar que las métricas en ambas bases tienen el tipo de `object`, lo cual sería correcto en python.

Sin embargo, se observa que en la base de **Analytics** hay algunos valores que no tienen el formato de fecha como *'2016+08*27'* y *'1082016'* y hay valores que se encuentran como string y otros no.

## Análisis Final

Debido a lo anterior, se procede a quitar la columna  `clientId` de la tabla **Analytics** dado que no tiene ningun valor, y a quitar los registros duplicados

In [18]:
df_ans2 = df_ans.drop(columns=['clientId'])

In [19]:
df_ans2.head()

,date,source,medium,campaign,adContent,operatingSystem,browser,language,country,city,sessionQualityDim,newVisits,pageviews,bounces
0,1082016,youtube.com,referral,(not set),NaN,Windows,Internet Explorer,not available in demo dataset,United States,not available in demo dataset,NaN,1.0,3.0,NaN
1,1082016,youtube.com,referral,(not set),NaN,Windows,Chrome,not available in demo dataset,United States,San Jose,NaN,NaN,3.0,NaN
2,1082016,youtube.com,referral,(not set),NaN,Macintosh,Chrome,not available in demo dataset,Philippines,Quezon City,NaN,1.0,3.0,NaN
3,1082016,youtube.com,referral,(not set),NaN,Windows,Firefox,not available in demo dataset,Canada,not available in demo dataset,NaN,1.0,4.0,NaN
4,1082016,youtube.com,referral,(not set),NaN,Windows,Internet Explorer,not available in demo dataset,United States,not available in demo dataset,NaN,1.0,2.0,NaN


In [20]:
df_ans2 = df_ans2.drop_duplicates()

In [21]:
df_ans.shape

(908148, 15)

In [22]:
df_ans.shape[0]-301268

606880

Como se puede observar se eliminan los duplicados y se queda con una base de 606880

Para dejar la base lista para el análsis final y la explotación de datos, se pueden eliminar todos los registros que tengan un valor Nulo, o si son númericos se puede reemplazar por el promedio de la lista, o el máximo o el mínimo.

En este caso se generaran los dos método en el cual en el segundo método se remplazara la columna por el promedio de los valores.

En cuanto a la columna `adContent` como no es númerica y no se puede ponderar, se eliminaran esos registros.

In [23]:
df_ans_notna = df_ans2.dropna()
df_ans_notna.head()

,date,source,medium,campaign,adContent,operatingSystem,browser,language,country,city,sessionQualityDim,newVisits,pageviews,bounces
833951,20170704,google,cpc,AW - Accessories,Google Merchandise Collection,iOS,Safari,not available in demo dataset,United States,not available in demo dataset,1.0,1.0,1.0,1.0
833964,20170704,google,cpc,AW - Accessories,{KeyWord:Google Branded Gear},Android,Chrome,not available in demo dataset,United States,not available in demo dataset,1.0,1.0,1.0,1.0
833981,20170704,google,cpc,AW - Accessories,Google Online Store,iOS,Safari,not available in demo dataset,United States,not available in demo dataset,1.0,1.0,1.0,1.0
833990,20170704,google,cpc,AW - Accessories,Google Merchandise Collection,Windows,Chrome,not available in demo dataset,United States,Sunnyvale,1.0,1.0,1.0,1.0
833994,20170704,google,cpc,AW - Accessories,Google Merchandise Collection,Android,Chrome,not available in demo dataset,United States,not available in demo dataset,1.0,1.0,1.0,1.0


In [24]:
df_ans_notna.shape

(50, 14)

In [25]:
df_ads_notna = df_ads.dropna()
df_ads_notna.head()

,_id,medium,support,reference,coverage,start_date,end_date,format,inversion,rating,channel,calculated,date_loaded
0,5ed306339be6f111072434b7,Medio_A,CUENTAME,C. Donación,FR,2020-05-19 13:21:49 UTC,2020-05-19 13:22:49 UTC,Mención 60,85.991836,0.0,2CAN+1,True,2020-05-31 01:19:47 UTC
1,5ed306339be6f11107242e49,Medio_A,CUENTAME,C. Restaurantes,FR,2020-05-14 13:57:53 UTC,2020-05-14 13:59:04 UTC,Mención 60,85.991836,0.0,2CAN+1,True,2020-05-31 01:19:47 UTC
2,5ed306339be6f11107242e2d,Medio_A,CUENTAME,C. Farmacia,FR,2020-05-11 13:57:59 UTC,2020-05-11 13:59:02 UTC,Mención 60,85.991836,0.0,2CAN+1,True,2020-05-31 01:19:47 UTC
3,5ed5d8ffc486d565172eae7a,Medio_A,CUENTAME,C. Pañales,FR,2020-05-27 14:03:35 UTC,2020-05-27 14:05:00 UTC,Mención 60,85.991836,0.0,2CAN+1,True,2020-06-02 04:43:43 UTC
4,5ed306339be6f11107243666,Medio_A,CUENTAME,D. de tema,FR,2020-05-20 14:00:21 UTC,2020-05-20 14:03:40 UTC,3 a 4 min,129.046442,0.0,2CAN+1,True,2020-05-31 01:19:47 UTC


In [26]:
df_ads_notna.shape

(73694, 13)

Como se observa si se utiliza la primera metodologia de quitar todos los registros que contengan un nulo, en la base de **Analytics** solo se tendrían 50 resgistros para analizar, por lo tanto no es recomendado

In [42]:
df_ans3 = df_ans2.dropna(subset=['adContent'])
df_ans3.shape

(9419, 14)

In [56]:
def replace_nan(df,col,int1=1):
    for i in col:
        if int1 == 1:
            df[i].fillna(value=int(df[i].mean(axis=0)), inplace=True)
        else:
            df[i].fillna(value=df[i].mean(axis=0), inplace=True)



In [57]:
col_ans = ['sessionQualityDim','newVisits','pageviews','bounces']

replace_nan(df_ans3,col_ans)
comp_analytics_mod = df_isnull(df_ans3)
comp_analytics_mod

C:\Users\Natalia\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,metrica,cant_comp,cant_null
0,date,9419,0
1,source,9419,0
2,medium,9419,0
3,campaign,9419,0
4,adContent,9419,0
5,operatingSystem,9419,0
6,browser,9419,0
7,language,9419,0
8,country,9419,0
9,city,9419,0


In [58]:
df_ads2 = df_ads.dropna(subset=['start_date'])
df_ads2.shape

(75229, 13)

In [62]:
col_ads = ['inversion']

replace_nan(df_ads2,col_ads,0)
comp_ads_mod = df_isnull(df_ads2)
comp_ads_mod

C:\Users\Natalia\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,metrica,cant_comp,cant_null
0,_id,75229,0
1,medium,75229,0
2,support,75229,0
3,reference,75229,0
4,coverage,75229,0
5,start_date,75229,0
6,end_date,75229,0
7,format,75229,0
8,inversion,75229,0
9,rating,75229,0


Como se observa no quedan valores nulos. Por lo tanto, ya se puede explotar la informacion de forma correcta
